In [1]:
import pandas as pd
import numpy as np

def clean_and_prep_data(input_file, output_file):
    # 1. Load Data
    print("Loading data...")
    try:
        df = pd.read_csv(input_file)
    except Exception as e:
        print(f"Error reading file: {e}")
        return

    # 2. Basic Filtering
    # FIX: Ensure we treat missing iso_codes as False so they don't crash the '~' operator
    # We use na=False so that NaN values are treated as not starting with OWID
    df = df[~df['iso_code'].str.startswith('OWID', na=False)]
    
    # Drop rows where location is International (if not already caught by iso_code check)
    df = df[df['location'] != 'International']

    # 3. Date formatting
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y', dayfirst=True, errors='coerce')
    
    # If format parsing failed (all NaT), try standard ISO format
    if df['date'].isna().all():
         df['date'] = pd.to_datetime(df['date'], errors='coerce')

    # 4. Handling Missing Numerical Values for Daily metrics
    # Fill NaN new_cases/deaths with 0 (assuming no report means 0)
    cols_to_zero = ['new_cases', 'new_deaths', 'total_cases', 'total_deaths']
    for col in cols_to_zero:
        if col in df.columns:
            df[col] = df[col].fillna(0)

    # 5. Feature Engineering (The "Tableau Ready" parts)
    
    # A. Survival Rate (Proxy for Recovery Performance)
    # Formula: (1 - (Total Deaths / Total Cases)) * 100
    # We handle division by zero using np.where
    df['Survival_Rate'] = np.where(
        df['total_cases'] > 0,
        (1 - (df['total_deaths'] / df['total_cases'])) * 100,
        0
    )

    # B. Mortality Rate (Case Fatality Rate)
    df['Mortality_Rate'] = np.where(
        df['total_cases'] > 0,
        (df['total_deaths'] / df['total_cases']) * 100,
        0
    )
    
    # C. Infection Rate per Capita
    # (Total Cases / Population) * 100
    df['Infection_Rate_Pct'] = np.where(
        df['population'] > 0,
        (df['total_cases'] / df['population']) * 100,
        0
    )

    # D. Economic Clusters (Binning GDP for easier filtering)
    # We treat GDP as static per country for this analysis
    # Using quantiles to create Low, Medium, High income buckets
    # Note: We calculate this based on the max GDP value per country to avoid fluctuation
    country_gdp = df.groupby('location')['gdp_per_capita'].max()
    # Create bins
    try:
        gdp_bins = pd.qcut(country_gdp, q=3, labels=['Low Income', 'Middle Income', 'High Income'])
        df['Income_Group'] = df['location'].map(gdp_bins)
    except:
        df['Income_Group'] = 'Unknown' # Fallback if GDP data is too sparse

    # 6. Final Clean & Export
    # Drop rows where vital correlation metrics are missing IF we want a pure correlation dataset,
    # but for a dashboard, we want to keep the dates.
    # We will just ensure the output is sorted.
    df = df.sort_values(by=['location', 'date'])

    print(f"Data cleaned. Rows: {len(df)}")
    print("Columns added: Survival_Rate, Mortality_Rate, Infection_Rate_Pct, Income_Group")
    
    df.to_csv(output_file, index=False)
    print(f"Successfully saved to {output_file}")

if __name__ == "__main__":
    # Ensure the input filename matches what is in your folder
    clean_and_prep_data('covid-data.csv', 'tableau_ready_covid_data.csv')

Loading data...
Data cleaned. Rows: 56499
Columns added: Survival_Rate, Mortality_Rate, Infection_Rate_Pct, Income_Group
Successfully saved to tableau_ready_covid_data.csv
